# What Is Intent Detection?
Intent detection is a task that identifies a user's intention or desired outcome from their query

---
# How can you build intent detection for our chatbot?
To build a reliable intent detection for your chatbot you need to cover 4 critical steps:

1. Defining intents
2. Setting up the intent detection prompt
3. Setting up handler logic prompts
4. Testing and evaluating prompts/models

---
# Write the intent detection prompt
```
Make sure to give clear directions, and follow best prompt engineering practices.

Here’s a simple example for the system prompt:



🤖 System Prompt:

You’re a LLM that detects intent from user queries. Your task is to classify the user's intent based on their query. Below are the possible intents with brief descriptions. Use these to accurately determine the user's goal, and output only the intent topic.

- Order Status: Inquiries about the current status of an order, including delivery tracking and estimated arrival times.

- Product Information: Questions regarding product details, specifications, availability, or compatibility.

- Payments: Queries related to making payments, payment methods, billing issues, or transaction problems.

- Returns: Requests or questions about returning a product, including return policies and procedures.

- Feedback: User comments, reviews, or general feedback about products, services, or experiences.

- Other: Choose this if the query doesn’t fall into any of the other intents.

💬 User Query:
I would like to check my last order.

🤖 Response:
Order status.

```


In [1]:
!pip install langchain langchain-openai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

In [12]:
examples = [
    {
        "query": "I would like to check my last order",
        "answer": "Order Status."
    }, {
        "query": "I am not able to make my payment via app",
        "answer": "Payments."
    }
]

In [13]:
example_template = """
Human : {query}
AI: {answer}
"""

In [14]:
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

In [15]:
prefix = """
  You’re a LLM that detects intent from user queries. Your task is to classify the user's intent based on their query. Below are the possible intents with brief descriptions. Use these to accurately determine the user's goal, and output only the intent topic.

  - Order Status: Inquiries about the current status of an order, including delivery tracking and estimated arrival times.

  - Product Information: Questions regarding product details, specifications, availability, or compatibility.

  - Payments: Queries related to making payments, payment methods, billing issues, or transaction problems.

  - Returns: Requests or questions about returning a product, including return policies and procedures.

  - Feedback: User comments, reviews, or general feedback about products, services, or experiences.

  - Other: Choose this if the query doesn’t fall into any of the other intents.

"""

In [16]:
suffix = """
Human : {query}
AI:
"""

In [23]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables=["query"],
    # example_selector='\n\n'
)

In [25]:
query = "I want to know a few details regarding the product"

print(few_shot_prompt_template.format(query=query))


  You’re a LLM that detects intent from user queries. Your task is to classify the user's intent based on their query. Below are the possible intents with brief descriptions. Use these to accurately determine the user's goal, and output only the intent topic.

  - Order Status: Inquiries about the current status of an order, including delivery tracking and estimated arrival times.

  - Product Information: Questions regarding product details, specifications, availability, or compatibility.

  - Payments: Queries related to making payments, payment methods, billing issues, or transaction problems.

  - Returns: Requests or questions about returning a product, including return policies and procedures.

  - Feedback: User comments, reviews, or general feedback about products, services, or experiences.

  - Other: Choose this if the query doesn’t fall into any of the other intents.




Human : I would like to check my last order
AI: Order Status.



Human : I am not able to make my paymen

In [26]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [27]:
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)

chain = LLMChain(
    llm=model,
    prompt=few_shot_prompt_template,
    verbose=True
)


In [28]:
query = "I want to know a few details regarding the product"
response = chain.run(query)



> Entering new LLMChain chain...
Prompt after formatting:

  You’re a LLM that detects intent from user queries. Your task is to classify the user's intent based on their query. Below are the possible intents with brief descriptions. Use these to accurately determine the user's goal, and output only the intent topic.

  - Order Status: Inquiries about the current status of an order, including delivery tracking and estimated arrival times.

  - Product Information: Questions regarding product details, specifications, availability, or compatibility.

  - Payments: Queries related to making payments, payment methods, billing issues, or transaction problems.

  - Returns: Requests or questions about returning a product, including return policies and procedures.

  - Feedback: User comments, reviews, or general feedback about products, services, or experiences.

  - Other: Choose this if the query doesn’t fall into any of the other intents.




Human : I would like to check my last order


In [29]:
response

'Product Information.'

In [30]:
query = "The product looks really bad, I am super disappointed"
response = chain.run(query)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

  You’re a LLM that detects intent from user queries. Your task is to classify the user's intent based on their query. Below are the possible intents with brief descriptions. Use these to accurately determine the user's goal, and output only the intent topic.

  - Order Status: Inquiries about the current status of an order, including delivery tracking and estimated arrival times.

  - Product Information: Questions regarding product details, specifications, availability, or compatibility.

  - Payments: Queries related to making payments, payment methods, billing issues, or transaction problems.

  - Returns: Requests or questions about returning a product, including return policies and procedures.

  - Feedback: User comments, reviews, or general feedback about products, services, or experiences.

  - Other: Choose this if the query doesn’t fall into any of the other intents.




Human : I would like to check my last order
